# Vessel Detection using Sentinel-1 Backscatter and Dynamic Compute

Here, we explore an exmaple of using Sentinel-1 backscatter imagery to identity vessel activity near Shanghai, China. 

### Methodology
In this notebook, we use the vertical polarized band, or [`vv`](https://www.earthdata.nasa.gov/data/platforms/space-based-platforms/sentinel-1) from Sentinel-1. The generally steps are as follows:
1. Input a start date and end date
2. Create a leaflet map over the area of interest
3. Create an [`ImageStack`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack) corresponding to all Sentinel-1 imagery over our input date range
4. Create a water mask using the mininum `vv` value across the temporal stack of imagery
5. Group the stack of imagery by date via [`ImageStack.groupby`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.groupby), for each date:
    * Mask to water
    * Mask `vv` band below 0 to identity vessel returns
    * Visualize the masked `vv` band

_Imports_

In [ ]:
import descarteslabs.dynamic_compute as dc

_Start and end dates_

In [ ]:
start_date = "2021-06-01"
end_date = "2021-07-01"

_Set up interactive map frame_

In [ ]:
m = dc.map
m.zoom = 13
m.center = 31.3739, 121.6455

Create an [`ImageStack`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack) using the sigma0 backscatter Product:

In [ ]:
sigma0 = dc.ImageStack.from_product_bands(
    "esa:sentinel-1:sigma0v:v1",
    "vv", 
    start_date, 
    end_date
)
vv = sigma0.pick_bands("vv")

Define the water mask and `visualize`:

In [ ]:
water_mask = vv.min(axis="images") > -12
water_mask.visualize("Water Mask", m)

In [ ]:
m

Group our stack by [`ImageStack.groupby`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.groupby):

In [ ]:
vv_grouped = vv.groupby(lambda image: (image.acquired.year, image.acquired.month, image.acquired.day)).max(axis="images")

For each date, mask to water and below 0, then finally `visualize`:

In [ ]:
for (year, month, day), vv_per_time in vv_grouped.groups.compute(m.geocontext()):
    (vv_per_time
     .mask(water_mask)
     .mask(vv_per_time < 0)
     .visualize(
         f"{year}-{month}-{day}",
         m,
         scales=[[-10, 10]],
         colormap="turbo",
         checkerboard=False,
     ))
    ## Uncomment below if you want to visualize the input VV band:
    # vv_per_time.mask(water_mask).visualize(f"VV {year}-{month}-{day}", m)